In [1]:
import numpy as np
import pandas as pd
import gc
import os, shutil
from tqdm import *
# import ipyvolume as ipv
import open3d as o3d

from multiprocessing import Pool, cpu_count
from struct import *
import time
from lidar_point_pb2 import *
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import argparse
import pickle
%matplotlib notebook


In [2]:
BASE_PATH = "/home/saby/Projects/ati/data/data/datasets/iisc/2020-02-10"
YELLI_FILE = "yelli_odo.csv"

STATIC_DATA_FOLDER = "2020-02-10-15-32-49-p15-auto-5loops_static"
DYNAMIC_2_DATA_FOLDER = "2020-02-10-15-57-38-p15-auto-5loops_dy_2"
DYNAMIC_1_DATA_FOLDER = "2020-02-10-16-12-42-p15-auto-5loops_dy_1"

STATIC_DATA_PATH = os.path.join(BASE_PATH, STATIC_DATA_FOLDER, "debug")
DYNAMIC_1_DATA_PATH = os.path.join(BASE_PATH, DYNAMIC_1_DATA_FOLDER, "debug")
DYNAMIC_2_DATA_PATH = os.path.join(BASE_PATH, DYNAMIC_2_DATA_FOLDER, "debug")

DATA_PATH = DYNAMIC_2_DATA_PATH
YELLI_PATH = os.path.join(DATA_PATH, YELLI_FILE)

In [3]:
PCD_FOLDER = 'complete_extracted'
PCD_PATH = os.path.join(DATA_PATH, PCD_FOLDER)
if not os.path.exists(PCD_PATH):
    os.makedirs(PCD_PATH)
else:
    shutil.rmtree(PCD_PATH)
    os.makedirs(PCD_PATH)

In [4]:
def getint(name):
    str_list = list(filter(str.isdigit, name))
    if len(str_list) == 1:
        return int(str_list[0])
    elif len(str_list) == 0:
        return 0
    else:
        print("String contains more than 1 integer")
        assert False

In [5]:
def read_packet(f):
    hdr = f.read(4)
    if hdr:
        #size, = struct.unpack("I", hdr)
        size, = unpack("I", hdr)
        # print(size) #Size is 488556
        #lp = ati.schema.LidarPoint()
        lp = LidarPoint()
        lp.ParseFromString(f.read(size))
        return lp

def read_frames(file_name):
    #file_name = "lidar-0"
    with open(file_name, "rb") as proto_file:
        while True:
            lp = read_packet(proto_file)

            if lp is None:
                break

            va = np.frombuffer(lp.vertical_angles, dtype=np.float64)
            ha = np.frombuffer(lp.horizontal_angles, dtype=np.float64)
            d = np.frombuffer(lp.distances, dtype=np.float64)
            i = np.frombuffer(lp.intensities, dtype=np.float64)

            beams = va.shape[0]
            num_points = d.shape[0]

            va = np.tile(va, num_points // beams).reshape(-1)
            ha = np.repeat(ha, beams).reshape(-1)
            d_v_cos = d * np.cos(va)

            frame = np.vstack([
                d_v_cos * np.sin(ha), # X
                d_v_cos * np.cos(ha), # Y
                d * np.sin(va),  # Z
                d, va, ha, i
            ]).T
            
            yield (lp.time, frame)

In [6]:
def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.show()
    elif where is 'mat_2d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.show()

In [7]:
def get_first_idx_offset(df):
    first_row = df.iloc[0]

    for idx, arg in enumerate(read_frames(os.path.join(DATA_PATH, 'lidar-0.pb'))):
        time = arg[0]
    #     print(idx, time)
        if time == first_row['time']:
            return int(first_row['frame_id'] - idx)
            break
        if idx == 100:
            print("No idx found upto {} frames".format(idx))
            assert False
    print("No idx found upto {} frames".format(idx))
    
def verify_first_idx_offset(idx_offset, df):
    first_row = df.iloc[0]
    for idx, arg in enumerate(read_frames(os.path.join(DATA_PATH, 'lidar-0.pb'))):
        if idx == int(first_row['frame_id']-idx_offset):
            return arg[0] == first_row['time']
        if idx == 100:
            print("No idx found upto {} frames".format(idx))
            assert False
    print("No idx found upto {} frames".format(idx))

In [8]:
df_yelli = pd.read_csv(YELLI_PATH)
first_idx_offset = get_first_idx_offset(df_yelli)
print("Found first idx offset to be {}".format(first_idx_offset))
assert verify_first_idx_offset(first_idx_offset, df_yelli)

Found first idx offset to be 6


In [9]:
def parallel_thread(args):
    frame_idx = args[0]
    time = args[1][0]
    frame = args[1][1]
    
    some_pcd = o3d.geometry.PointCloud()
    some_arr = frame[:,:3]
    some_arr = some_arr[frame[:,3] != 0]
    some_pcd.points = o3d.utility.Vector3dVector(some_arr)
    
    single_pcd_fname = str(pb_offset + frame_idx + first_idx_offset) + ".pcd"
    single_pcd_path = os.path.join(DATA_PATH, 'complete_extracted', single_pcd_fname)
    o3d.io.write_point_cloud(single_pcd_path, some_pcd, write_ascii=True)
    gc.collect()
    return frame_idx

In [10]:
pb_offset = 0
pb_files = sorted([file for file in os.listdir(DATA_PATH) if file[:5] == 'lidar' and file.endswith(".pb")], key=getint)

for pb_file in pb_files:
    print('Reading frames from {}'.format(pb_file))
    parallel_args = enumerate(read_frames(os.path.join(DATA_PATH, pb_file)))
    process_pool = Pool(cpu_count()-1)
    fidx = [each for each in tqdm_notebook(process_pool.imap_unordered(parallel_thread, parallel_args))]#,
                                                 #total = len(parallel_args))]
    process_pool.terminate()
    gc.collect()
    
    pb_offset += max(fidx)+1
    
    print('{} converted to {} pcd files'.format(pb_file, max(fidx)+1))
    print("****************************************************************")
print("First pcd starts at {}.pcd".format(first_idx_offset))
print("Last pcd ends at {}.pcd".format(first_idx_offset+pb_offset-1))
print("Total no of pcds created: {}".format(pb_offset))


Reading frames from lidar-0.pb


/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



lidar-0.pb converted to 5994 pcd files
****************************************************************
Reading frames from lidar-1.pb



lidar-1.pb converted to 1549 pcd files
****************************************************************
First pcd starts at 6.pcd
Last pcd ends at 7548.pcd
Total no of pcds created: 7543


In [11]:
some_pcd = o3d.io.read_point_cloud(os.path.join(DATA_PATH, 'complete_extracted', "50.pcd"))
draw_pcd(some_pcd, 'opn_nb')

JVisualizer with 1 geometries